In [ ]:
'''%pip install mediapipe
%pip install opencv-python
%pip install numpy
%pip install pycaw'''

In [11]:
#!pip install autopy
import cv2
import autopy
import mediapipe as mp
from math import sqrt
from ctypes import cast, POINTER
from comtypes import CLSCTX_ALL
from pycaw.pycaw import AudioUtilities, IAudioEndpointVolume

In [ ]:
hands = mp.solutions.hands.Hands(static_image_mode = False, max_num_hands = 1, min_tracking_confidence = 0.5, min_detection_confidence = 0.5)

cap = cv2.VideoCapture(0)

mpDraw = mp.solutions.drawing_utils

width, height = autopy.screen.size()

hands_size = 3 #размер руки (3 - норма)

sensetivity = 2 #чувствительмость курсора (чем выше тем меньше надо двигать рукой. 1 - норма)

devices = AudioUtilities.GetSpeakers()
interface = devices.Activate(IAudioEndpointVolume._iid_, CLSCTX_ALL, None)
volume = cast(interface, POINTER(IAudioEndpointVolume))
volRange = volume.GetVolumeRange()
volume.SetMasterVolumeLevel(0, None)
minVol = volRange[0]
maxVol = volRange[1]

In [4]:
def point_landmark(point_number, processed_img, hand) :
    for id, lm in enumerate(processed_img.multi_hand_landmarks[hand].landmark) :
        if id == point_number :
            return lm
        
def dist(x1, y1, x2, y2) :
    dx = abs(x1 - x2)
    dy = abs(y1 - y2)
    dist = sqrt(dx**2 + dy**2)
    return dist

In [5]:
dist(0, 0, sqrt(2), sqrt(2))

2.0

In [10]:
while True :    
    _, img = cap.read()
    result = hands.process(img)
    #print(result)
    if result.multi_hand_landmarks :

        h, w, _ = img.shape
        '''h *= 1.2
        w *= 1.2'''
        p6_lm = point_landmark(6, result, 0)
        p4_lm = point_landmark(4, result, 0)
        p8_lm = point_landmark(8, result, 0)
        p0_lm = point_landmark(0, result, 0)
        p7_lm = point_landmark(7, result, 0)
        p11_lm = point_landmark(11, result, 0)
        p15_lm = point_landmark(15, result, 0)
        p19_lm = point_landmark(19, result, 0)
        p17_lm = point_landmark(17, result, 0)
        p20_lm = point_landmark(20, result, 0)

        p6_x, p6_y = int(p6_lm.x * w), int(p6_lm.y * h)
        p4_x, p4_y = int(p4_lm.x * w), int(p4_lm.y * h)
        p0_x, p0_y = int(p0_lm.x * w), int(p0_lm.y * h)
        p8_x, p8_y = int(p8_lm.x * w), int(p8_lm.y * h)
        p7_x, p7_y = int(p7_lm.x * w), int(p7_lm.y * h)
        p11_x, p11_y = int(p11_lm.x * w), int(p11_lm.y * h)
        p15_x, p15_y = int(p15_lm.x * w), int(p15_lm.y * h)
        p19_x, p19_y = int(p19_lm.x * w), int(p19_lm.y * h)
        p17_x, p17_y = int(p17_lm.x * w), int(p17_lm.y * h)
        p20_x, p20_y = int(p20_lm.x * w), int(p20_lm.y * h)

        if ((dist(p7_x, p7_y, p11_x, p11_y) <= 15*hands_size) and (dist(p11_x, p11_y, p15_x, p15_y) <= 15*hands_size) and (dist(p15_x, p15_y, p19_x, p19_y) <= 15*hands_size)) and ((dist(p0_x, p0_y, p7_x, p7_y) <= 40*hands_size) and (dist(p0_x, p0_y, p11_x, p11_y) <= 30*hands_size) and (dist(p0_x, p0_y, p15_x, p15_y) <= 25*hands_size) and (dist(p0_x, p0_y, p19_x, p19_y) <= 30*hands_size)) :
            mouse_button_press = True
        else :
            mouse_button_press = False
        
        if (dist(p4_x, p4_y, p6_x, p6_y) <= 7*hands_size) : 
            '''and ((dist(p7_x, p7_y, p11_x, p11_y) <= 15*hands_size) and (dist(p11_x, p11_y, p15_x, p15_y) <= 15*hands_size) and (dist(p15_x, p15_y, p19_x, p19_y) <= 15*hands_size)) and ((dist(p0_x, p0_y, p7_x, p7_y) <= 40*hands_size) and (dist(p0_x, p0_y, p11_x, p11_y) <= 30*hands_size) and (dist(p0_x, p0_y, p15_x, p15_y) <= 25*hands_size) and (dist(p0_x, p0_y, p19_x, p19_y) <= 30*hands_size))'''
            mouse_button_click = True
        else :
            mouse_button_click = False
        
        if True :
            volume.SetMasterVolumeLevelScalar(dist(p4_lm.x, p4_lm.y, p8_lm.x, p8_lm.y), None)

        for id, lm in enumerate(result.multi_hand_landmarks[0].landmark) :
            h, w, _ = img.shape
            '''h *= 1.2
            w *= 1.2''' 
            cx, cy = int(lm.x*w), int(lm.y*h)
            #cv2.circle(img, (cx, cy), 3, (255, 0, 255))
            #print(id, lm)
            if id == 9 :
                #cv2.circle(img, (cx, cy), 15, (255, 255, 255), cv2.FILLED)
                #cx, cy = 
                dx = cx - w/2
                dy = cy - h/2
                mouse_x = width - (width/2 + dx * width / w * sensetivity)
                mouse_y = height/2 + dy * height / h * sensetivity
                if 0 < mouse_x < width and 0 < mouse_y < height :
                    print(dist(p7_x, p7_y, p11_x, p11_y))
                    if mouse_button_press == True and mouse_button_click != True :
                        autopy.mouse.toggle(autopy.mouse.Button.LEFT, True)
                        autopy.mouse.move(mouse_x, mouse_y)
                    else :
                        autopy.mouse.toggle(autopy.mouse.Button.LEFT, False)
                        autopy.mouse.move(mouse_x, mouse_y)
                    if mouse_button_click == True :
                        autopy.mouse.click(autopy.mouse.Button.LEFT, 0.9)
                        autopy.mouse.move(mouse_x, mouse_y)
                    else :
                        autopy.mouse.move(mouse_x, mouse_y)

        mpDraw.draw_landmarks(img, result.multi_hand_landmarks[0], mp.solutions.hands.HAND_CONNECTIONS)
    #cv2.imshow("Hand image", img)
    cv2.waitKey()

50.695167422546305
43.08131845707603
52.392747589718944
53.71219600798314
55.54277630799526
55.154328932550705
54.78138369920935
52.92447448959697
52.20153254455275
52.92447448959697
50.92150822589606
51.62363799656123
50.92150822589606
51.54609587543949
51.86520991955976
51.86520991955976
51.54609587543949
50.0
50.96076922496363
50.695167422546305
48.507731342539614
48.27007354458868
48.507731342539614
48.507731342539614
47.29693436154187
46.32493928760188
47.29693436154187
46.32493928760188
46.32493928760188
46.09772228646444
47.29693436154187
48.05205510693585
48.05205510693585
48.05205510693585
47.853944456021594
47.853944456021594
49.03060268852505
48.83646178829912
48.83646178829912
49.8196748283246
48.83646178829912
48.83646178829912
49.8196748283246
50.99019513592785
50.99019513592785
51.19570294468082
50.99019513592785
50.00999900019995
51.19570294468082
50.00999900019995
51.19570294468082
51.419840528729765
51.19570294468082
51.419840528729765
51.19570294468082
51.19570294468

KeyboardInterrupt: 